In [1]:
import numpy as np                                                             # numerical calculation
import matplotlib.pyplot as plt                                                # plotting
import tensorflow as tf                                                        # load neural network lib
from tensorflow.keras.initializers import Initializer                          # initiation for RBF kernels
from tensorflow.keras.layers import Layer                                      # different layers provided by keras
from tensorflow.keras.initializers import RandomUniform, Initializer, Constant # tensorflow initializers

In [2]:
# create centers given two dimensional peaks

# Peak generator
class CustomPeaks(Initializer):
    def __init__(self, mu1, mu2):
        # save the centers
        self.mu1 = mu1
        self.mu2 = mu2
        super().__init__()
    
    def __call__(self, shape, dtype=None):
        # combine and return centers as float
        outs = np.c_[self.mu1, self.mu2]
        return tf.convert_to_tensor(outs, dtype="float")

# select the centers randomly
class InitCentersRandom(Initializer):
    """ Initializer for initialization of centers of RBF network
        as random samples from the given data set.

    # Arguments
        X: matrix, dataset to choose the centers from (random rows
          are taken as centers)
    """

    def __init__(self, X):
        self.X = X
        super().__init__()

    def __call__(self, shape, dtype=None):
        assert shape[1:] == self.X.shape[1:]  # check dimension

        # np.random.randint returns ints from [low, high) !
        idx = np.random.randint(self.X.shape[0], size=shape[0])

        return self.X[idx, :]


# rbf layer definition
class RBFLayer(Layer):
    """ Layer of Gaussian RBF units.

    # Example

    ```python
        model = Sequential()
        model.add(RBFLayer(10,
                           initializer=InitCentersRandom(X),
                           betas=1.0,
                           input_shape=(1,)))
        model.add(Dense(1))
    ```


    # Arguments
        output_dim: number of hidden units (i.e. number of outputs of the
                    layer)
        initializer: instance of initiliazer to initialize centers
        betas: float, initial value for betas

    """

    def __init__(self, output_dim, initializer=None, betas=1.0, **kwargs):

        self.output_dim = output_dim

        # betas is either initializer object or float
        if isinstance(betas, Initializer):
            self.betas_initializer = betas
        else:
            self.betas_initializer = Constant(value=betas)

        # of there is no initializer use the RandomUniform function
        self.initializer = initializer if initializer else RandomUniform(
            0.0, 1.0)

        super().__init__(**kwargs)

    # builds the layer 
    def build(self, input_shape):

        # add the center and bias weights using inits
        self.centers = self.add_weight(name='centers',
                                       shape=(self.output_dim, input_shape[1]),
                                       initializer=self.initializer,
                                       trainable=False)
        self.betas = self.add_weight(name='betas',
                                     shape=(self.output_dim,),
                                     initializer=self.betas_initializer,
                                     # initializer='ones',
                                     trainable=False)

        super().build(input_shape)

    # this is called when the RBF layer is used on data
    def call(self, x):

        C = tf.expand_dims(self.centers, -1)  # inserts a dimension of 1
        H = tf.transpose(C-tf.transpose(x))  # matrix of differences
        return tf.exp(-self.betas * tf.math.reduce_sum(H**2, axis=1))

    # calculate outputshape based on input shape
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    # have to define get_config to be able to use model_from_json
    def get_config(self):
        config = {
            'output_dim': self.output_dim
        }
        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [9]:
def get_model():
    # seq model
    model = tf.keras.models.Sequential()
    model.add(rbflayer)
    # classification layer
    model.add(tf.keras.layers.Dense(units=2, activation='linear'))
    #model.add(tf.keras.layers.Dense(units=1, activation="sigmoid", kernel_initializer="he_normal", name="Final_dense"))
    # compile with mse loss and sgd optimizer
    model.compile(optimizer='sgd', loss='mean_squared_error')
    model.summary()
    return model

In [10]:
# XOR DATA
x1 = np.array([0, 0, 1, 1])
x2 = np.array([0, 1, 0, 1])
X = np.c_[x1, x2]
X = X.astype("float") # convert type to float for tensorflow

# create labels
T = np.array([0, 1, 1, 0])
T = T.astype("float")

print(X.shape)
print(T.shape)

(4, 2)
(4,)


In [11]:
# Create RBF using tensorflow***

# centers initiation for 2 neurons
mu1 = np.array([0, 1])
mu2 = np.array([1, 0])

# create the rbf layer from the class we created above
rbflayer = RBFLayer(2, initializer=CustomPeaks(mu1, mu2), betas=0.5, input_shape=(2,), name="RFB_layer")

# see network params
net = get_model()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
RFB_layer (RBFLayer)         (None, 2)                 6         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 6         
Total params: 12
Trainable params: 6
Non-trainable params: 6
_________________________________________________________________


In [12]:
# do training
net.fit(X, T, epochs=20, verbose=1)

Epoch 1/20
1/1 [==============================] - 0s 280ms/step - loss: 0.9406
Epoch 2/20
1/1 [==============================] - 0s 3ms/step - loss: 0.9162
Epoch 3/20
1/1 [==============================] - 0s 6ms/step - loss: 0.8926
Epoch 4/20
1/1 [==============================] - 0s 3ms/step - loss: 0.8699
Epoch 5/20
1/1 [==============================] - 0s 4ms/step - loss: 0.8481
Epoch 6/20
1/1 [==============================] - 0s 8ms/step - loss: 0.8270
Epoch 7/20
1/1 [==============================] - 0s 5ms/step - loss: 0.8067
Epoch 8/20
1/1 [==============================] - 0s 8ms/step - loss: 0.7871
Epoch 9/20
1/1 [==============================] - 0s 5ms/step - loss: 0.7683
Epoch 10/20
1/1 [==============================] - 0s 4ms/step - loss: 0.7501
Epoch 11/20
1/1 [==============================] - 0s 4ms/step - loss: 0.7326
Epoch 12/20
1/1 [==============================] - 0s 9ms/step - loss: 0.7157
Epoch 13/20
1/1 [==============================] - 0s 4ms/step - loss: 

In [13]:
net.evaluate(X, T) # Training Loss

1/1 [==============================] - 0s 100ms/step - loss: 0.5887


0.5887014269828796

In [14]:
net.predict(X)

array([[-0.13113295,  0.06940764],
       [-0.4588065 , -0.02935679],
       [ 0.12240374,  0.15874031],
       [-0.13113295,  0.06940764]], dtype=float32)

In [15]:
# create the meshgrid with rules from the matlab code

meshsteps = np.linspace(start=np.min(X)-3*Spread, stop=np.max(X)+3*Spread,
                        num=861, retstep=False)
x1, x2 = np.meshgrid(meshsteps, meshsteps)

XX = np.array([x1.flatten(),
               x2.flatten()])
print(XX.shape)
# reshape into transpose which is python format for neural networks
XX = XX.T
print(XX.shape)

ValueError: cannot reshape array of size 20000 into shape (100,100)